In [1]:
import pandas as pd
import numpy as np

In [2]:
from tqdm import tqdm
import pickle
import re


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import inaugural, stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/meghanaasok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/meghanaasok/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/meghanaasok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [5]:
label = pd.read_csv(r"/Users/meghanaasok/Downloads/mlrw-biomedicalhackathon/labelled_train_data.csv")

In [6]:
label.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'ctrl', 'pert',
       'channel_count', 'characteristics_ch1', 'contact_address',
       'contact_city', 'contact_country', 'contact_department',
       'contact_email', 'contact_institute', 'contact_name', 'contact_state',
       'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'growth_protocol_ch1', 'hyb_protocol',
       'label_ch1', 'label_protocol_ch1', 'last_update_date', 'molecule_ch1',
       'organism_ch1', 'platform_id', 'scan_protocol', 'source_name_ch1',
       'status', 'submission_date', 'supplementary_file', 'taxid_ch1', 'title',
       'treatment_protocol_ch1', 'type', 'contact_phone', 'contact_laboratory',
       'relation', 'contact_fax', 'biomaterial_provider_ch1',
       'contact_web_link', 'characteristics_ch2', 'extract_protocol_ch2',
       'label_ch2', 'label_protocol_ch2', 'molecule_ch2', 'organism_ch2',
       'source_name_ch2', 'taxid_ch2', 'treatment_protocol_ch2',
       'biom

In [7]:
label.isna().sum()

Unnamed: 0                    0
geo_accession                 0
gse_id                        0
ctrl                          0
pert                          0
channel_count                 0
characteristics_ch1           0
contact_address               0
contact_city                  0
contact_country               0
contact_department          120
contact_email               229
contact_institute             0
contact_name                  0
contact_state               174
data_processing               0
data_row_count                0
description                 147
extract_protocol_ch1          0
growth_protocol_ch1         341
hyb_protocol                110
label_ch1                   110
label_protocol_ch1          110
last_update_date              0
molecule_ch1                  0
organism_ch1                  0
platform_id                   0
scan_protocol               110
source_name_ch1               0
status                        0
submission_date               0
suppleme

In [8]:
label=label.drop(['contact_address','contact_department', 'contact_email',
       'contact_institute', 'contact_name','last_update_date','contact_fax','submission_date',
       'contact_phone', 'contact_laboratory','relation','contact_web_link','status','pert'],axis=1)

In [9]:
label.isna().sum()

Unnamed: 0                    0
geo_accession                 0
gse_id                        0
ctrl                          0
channel_count                 0
characteristics_ch1           0
contact_city                  0
contact_country               0
contact_state               174
data_processing               0
data_row_count                0
description                 147
extract_protocol_ch1          0
growth_protocol_ch1         341
hyb_protocol                110
label_ch1                   110
label_protocol_ch1          110
molecule_ch1                  0
organism_ch1                  0
platform_id                   0
scan_protocol               110
source_name_ch1               0
supplementary_file          110
taxid_ch1                     0
title                         0
treatment_protocol_ch1      325
type                          0
biomaterial_provider_ch1    609
characteristics_ch2         597
extract_protocol_ch2        597
label_ch2                   597
label_pr

In [10]:
unlabel=pd.read_csv(r"/Users/meghanaasok/Downloads/mlrw-biomedicalhackathon/unlabelled_train_data.csv")

In [11]:
unlabel=unlabel.drop(['contact_address','contact_department', 'contact_email',
       'contact_institute', 'contact_name','last_update_date','contact_fax','submission_date',
       'contact_phone', 'contact_laboratory','relation','contact_web_link','status'],axis=1)

In [12]:
unlabel.isna().sum()

Unnamed: 0                      0
geo_accession                   0
gse_id                          0
channel_count                   0
characteristics_ch1           136
contact_city                   27
contact_country                26
contact_state                6463
data_processing               153
data_row_count                 31
description                  5318
extract_protocol_ch1          348
growth_protocol_ch1          7520
hyb_protocol                 6066
label_ch1                    5870
label_protocol_ch1           6066
molecule_ch1                   10
organism_ch1                    0
platform_id                    22
scan_protocol                6068
source_name_ch1                 0
supplementary_file           5736
taxid_ch1                      15
title                           0
treatment_protocol_ch1       7600
type                            0
biomaterial_provider_ch1    18991
characteristics_ch2         18255
extract_protocol_ch2        18418
label_ch2     

In [13]:
unlabel.shape

(19456, 38)

In [14]:
missing = label.isna().sum()/len(label)
missing2 = unlabel.isna().sum()/len(unlabel)


In [15]:
missing

Unnamed: 0                  0.000000
geo_accession               0.000000
gse_id                      0.000000
ctrl                        0.000000
channel_count               0.000000
characteristics_ch1         0.000000
contact_city                0.000000
contact_country             0.000000
contact_state               0.279294
data_processing             0.000000
data_row_count              0.000000
description                 0.235955
extract_protocol_ch1        0.000000
growth_protocol_ch1         0.547352
hyb_protocol                0.176565
label_ch1                   0.176565
label_protocol_ch1          0.176565
molecule_ch1                0.000000
organism_ch1                0.000000
platform_id                 0.000000
scan_protocol               0.176565
source_name_ch1             0.000000
supplementary_file          0.176565
taxid_ch1                   0.000000
title                       0.000000
treatment_protocol_ch1      0.521669
type                        0.000000
b

In [16]:
missing2

Unnamed: 0                  0.000000
geo_accession               0.000000
gse_id                      0.000000
channel_count               0.000000
characteristics_ch1         0.006990
contact_city                0.001388
contact_country             0.001336
contact_state               0.332185
data_processing             0.007864
data_row_count              0.001593
description                 0.273335
extract_protocol_ch1        0.017887
growth_protocol_ch1         0.386513
hyb_protocol                0.311780
label_ch1                   0.301706
label_protocol_ch1          0.311780
molecule_ch1                0.000514
organism_ch1                0.000000
platform_id                 0.001131
scan_protocol               0.311883
source_name_ch1             0.000000
supplementary_file          0.294819
taxid_ch1                   0.000771
title                       0.000000
treatment_protocol_ch1      0.390625
type                        0.000000
biomaterial_provider_ch1    0.976100
c

In [17]:
label = label.loc[:, missing < 0.5]
unlabel = unlabel.loc[:, missing2 < 0.5]

In [18]:
label.shape

(623, 25)

In [19]:
unlabel.shape

(19456, 26)

In [20]:
label.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'ctrl', 'channel_count',
       'characteristics_ch1', 'contact_city', 'contact_country',
       'contact_state', 'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'hyb_protocol', 'label_ch1',
       'label_protocol_ch1', 'molecule_ch1', 'organism_ch1', 'platform_id',
       'scan_protocol', 'source_name_ch1', 'supplementary_file', 'taxid_ch1',
       'title', 'type'],
      dtype='object')

In [21]:
unlabel.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'channel_count',
       'characteristics_ch1', 'contact_city', 'contact_country',
       'contact_state', 'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'growth_protocol_ch1', 'hyb_protocol',
       'label_ch1', 'label_protocol_ch1', 'molecule_ch1', 'organism_ch1',
       'platform_id', 'scan_protocol', 'source_name_ch1', 'supplementary_file',
       'taxid_ch1', 'title', 'treatment_protocol_ch1', 'type'],
      dtype='object')

In [22]:
unlabel=unlabel.drop(['growth_protocol_ch1','treatment_protocol_ch1'],axis=1)

In [23]:
def preprocess(data_df):
    data_df['cleaned_feature'] = ''
    
    # Initializing Stopwords and Lemmatization objects
    stop_words = set(stopwords.words('english'))
    wordnet_lemm = WordNetLemmatizer()
    
    # Pattern to detect characters which are not alphabets or numbers so they can removed
    alpha_or_numeric = "[^a-zA-Z0-9- ]"

    for index, row in tqdm(data_df.iterrows(), total=data_df.shape[0]):
    
        sample = row['feature']
        pre_txt = re.sub(alpha_or_numeric, " ", sample)
        pre_txt = sample.lower()
            
        
        # Removing stop words and lemmatizing different words in preprocessed text and making the final processed text
        sample_words = [wordnet_lemm.lemmatize(w) for w in pre_txt.split() if w not in stop_words and len(w)>1]
        pre_proc_ver = ' '.join(sample_words)
        
        data_df.loc[index, 'cleaned_feature'] = pre_proc_ver
        
    return data_df
        

In [24]:
cols = ['gse_id','contact_city','contact_country','contact_state','organism_ch1','platform_id','extract_protocol_ch1']

In [25]:
columns = ['gse_id','contact_city','contact_country','contact_state','organism_ch1','platform_id','extract_protocol_ch1','characteristics_ch1','data_processing','hyb_protocol','label_ch1	label_protocol_ch1',	'molecule_ch1',	'scan_protocol',	'source_name_ch1,','title'] # choose suitable set of columns according to your needs

label.loc[:, 'feature'] = ""

# We are concatenating all the columns with a space.
label['feature'] = label.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)

In [26]:
label['feature']

0      GSE66250 Wuerzburg Germany nan Homo sapiens GP...
1      GSE66250 Wuerzburg Germany nan Homo sapiens GP...
2      GSE66250 Wuerzburg Germany nan Homo sapiens GP...
3      GSE66250 Wuerzburg Germany nan Homo sapiens GP...
4      GSE52505 Seoul South Korea nan Homo sapiens GP...
                             ...                        
618    GSE59980 Los Angeles USA CA Homo sapiens GPL11...
619    GSE59980 Los Angeles USA CA Homo sapiens GPL11...
620    GSE59980 Los Angeles USA CA Homo sapiens GPL11...
621    GSE59980 Los Angeles USA CA Homo sapiens GPL11...
622    GSE59980 Los Angeles USA CA Homo sapiens GPL11...
Name: feature, Length: 623, dtype: object

In [27]:
clean_label=preprocess(label)

100%|████████████████████████████████████████| 623/623 [00:01<00:00, 603.71it/s]


In [28]:
clean_label2=clean_label.copy()

In [29]:
clean_label2=clean_label2.drop(['ctrl'],axis=1)

In [30]:
stop_words = set(stopwords.words('english'))
vect = TfidfVectorizer(analyzer="word", preprocessor=None, stop_words=stop_words, max_features=10000)


X_train_vect = vect.fit_transform(clean_label2['cleaned_feature'])
y_train1 = clean_label['ctrl'].astype(int).tolist()

In [31]:
logistic_reg = LogisticRegression(max_iter=1000)
logistic_reg.fit(X_train_vect, y_train1)

LogisticRegression(max_iter=1000)

In [33]:
print("Classification report for ctrl (1 - ctrl, 0 - pert)\n")
y_predictions = logistic_reg.predict(X_train_vect)
print(classification_report(y_train1, y_predictions))

Classification report for ctrl (1 - ctrl, 0 - pert)

              precision    recall  f1-score   support

           0       0.66      0.74      0.70       319
           1       0.69      0.61      0.64       304

    accuracy                           0.67       623
   macro avg       0.68      0.67      0.67       623
weighted avg       0.68      0.67      0.67       623



In [34]:
unlabel.loc[:, 'feature'] = ""

# We are concatenating all the columns with a space.
unlabel['feature'] = unlabel.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)

In [35]:
clean_unlabel = preprocess(unlabel.copy())
X_trainun_vect = vect.transform(clean_unlabel['cleaned_feature'].tolist())



100%|███████████████████████████████████| 19456/19456 [00:08<00:00, 2389.87it/s]


In [36]:
X_trainun_vect

<19456x977 sparse matrix of type '<class 'numpy.float64'>'
	with 392289 stored elements in Compressed Sparse Row format>

In [37]:
clean_label2.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'channel_count',
       'characteristics_ch1', 'contact_city', 'contact_country',
       'contact_state', 'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'hyb_protocol', 'label_ch1',
       'label_protocol_ch1', 'molecule_ch1', 'organism_ch1', 'platform_id',
       'scan_protocol', 'source_name_ch1', 'supplementary_file', 'taxid_ch1',
       'title', 'type', 'feature', 'cleaned_feature'],
      dtype='object')

In [38]:
clean_label2.shape

(623, 26)

In [39]:
clean_unlabel.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'channel_count',
       'characteristics_ch1', 'contact_city', 'contact_country',
       'contact_state', 'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'hyb_protocol', 'label_ch1',
       'label_protocol_ch1', 'molecule_ch1', 'organism_ch1', 'platform_id',
       'scan_protocol', 'source_name_ch1', 'supplementary_file', 'taxid_ch1',
       'title', 'type', 'feature', 'cleaned_feature'],
      dtype='object')

In [40]:
clean_unlabel.shape

(19456, 26)

In [41]:
y_predun=logistic_reg.predict(X_trainun_vect)

In [42]:
y_predun

array([1, 1, 1, ..., 0, 0, 0])

In [43]:
result = pd.DataFrame(y_predun, columns = ['ctrl'])

In [44]:
result.head

<bound method NDFrame.head of        ctrl
0         1
1         1
2         1
3         1
4         1
...     ...
19451     0
19452     0
19453     0
19454     0
19455     0

[19456 rows x 1 columns]>

In [45]:
X_train_temp=pd.concat([clean_unlabel,result],axis=1,join='inner')

In [46]:
X_train_final=pd.concat([clean_label,X_train_temp])

In [47]:
X_train_final.shape

(20079, 27)

In [48]:
X_train_final.columns

Index(['Unnamed: 0', 'geo_accession', 'gse_id', 'ctrl', 'channel_count',
       'characteristics_ch1', 'contact_city', 'contact_country',
       'contact_state', 'data_processing', 'data_row_count', 'description',
       'extract_protocol_ch1', 'hyb_protocol', 'label_ch1',
       'label_protocol_ch1', 'molecule_ch1', 'organism_ch1', 'platform_id',
       'scan_protocol', 'source_name_ch1', 'supplementary_file', 'taxid_ch1',
       'title', 'type', 'feature', 'cleaned_feature'],
      dtype='object')

In [49]:
from sklearn.model_selection import train_test_split


In [50]:
X=X_train_final.drop(['ctrl'],axis=1)
y=X_train_final['ctrl']

In [51]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2)

In [52]:
X_train_vect_final = vect.transform(X_train['cleaned_feature'].tolist())
X_val_vect=vect.transform(X_val['cleaned_feature'].tolist())

In [53]:
log_reg=LogisticRegression(max_iter=1000)

In [54]:
log_reg.fit(X_train_vect_final,y_train)

LogisticRegression(max_iter=1000)

In [55]:
y_pred1=log_reg.predict(X_val_vect)

In [56]:
print(classification_report(y_val, y_pred1))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      2497
         1.0       0.96      0.95      0.96      1519

    accuracy                           0.97      4016
   macro avg       0.97      0.96      0.96      4016
weighted avg       0.97      0.97      0.97      4016



In [57]:
test = pd.read_csv(r'/Users/meghanaasok/Downloads/mlrw-biomedicalhackathon/data_only_test.csv')

test.loc[:, 'feature'] = ""
test['feature'] = test.apply(lambda row: ' '.join([str(row[i]) for i in cols]), axis=1)

test.drop(columns = ['contact_address','contact_department', 'contact_email',
       'contact_institute', 'contact_name','last_update_date','contact_fax','submission_date',
       'contact_phone', 'contact_laboratory','relation','contact_web_link','status','supplementary_file'], inplace = True)

cleaned_test_df = preprocess(test.copy())
X_test_vect = vect.transform(cleaned_test_df['cleaned_feature'].tolist())

test_predictions = log_reg.predict(X_test_vect)
cleaned_test_df['ctrl'] = test_predictions

cleaned_test_df = cleaned_test_df[['geo_accession', 'ctrl']]

# make sure its a float!
cleaned_test_df['ctrl'] = cleaned_test_df['ctrl'].astype(np.float64)

cleaned_test_df.to_csv('submission.csv', index=False)

100%|█████████████████████████████████████| 6070/6070 [00:01<00:00, 3740.66it/s]
